In [ ]:
import os
import zipfile
import numpy as np
import paddle
from paddle.io import Dataset, DataLoader

In [4]:
train_parameters = {
    "batch_size":16,
    "epochs":100
}

In [ ]:
# 构建类别映射表，将语言标签映射为数字id
def build_class_map(gt_file_path):
    languages = set()  # 用于存储所有出现过的语言标签
    with open(gt_file_path, encoding='utf-8') as f:
        for line in f:
            parts = line.strip().split(',')
            if len(parts) >= 2:
                languages.add(parts[1])  # 收集语言标签
    
    # 按字母顺序排序语言标签，并分配唯一的数字id
    return {lang: idx for idx, lang in enumerate(sorted(languages))}

# 调用函数，生成训练集的语言类别映射表
class_map = build_class_map('data/train/gt.txt')

In [ ]:
def unzip_data(zip_path, target_path):
    os.makedirs(target_path, exist_ok=True)
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(target_path)

# 替换为相应的路径
train_gt_zip_path = "data/data336725/ch8_training_word_gt_v2.zip"
train_images_zip_prefix = "data/data336725/ch8_training_word_images_gt_part_"
eval_gt_zip_path = "data/data336725/ch8_validation_word_gt_v2.zip"
eval_images_zip_path = "data/data336725/ch8_validation_word_images_gt.zip"

train_target_path = "data/train"
eval_target_path = "data/eval"

# 解压训练集标注
unzip_data(train_gt_zip_path, train_target_path)

# 解压训练集图片（多个分包）
for i in range(1, 3):
    zip_path = f"{train_images_zip_prefix}{i}.zip"
    unzip_data(zip_path, train_target_path)

# 解压验证集标注和图片
unzip_data(eval_gt_zip_path, eval_target_path)
unzip_data(eval_images_zip_path, eval_target_path)

In [ ]:
class MultilingualDataset(Dataset):
    """
    多语言数据集类，用于加载带有语言标签和坐标信息的图片样本。

    参数:
        gt_file (str): 标注文件路径，包含图片文件名和语言标签。
        coord_file (str): 坐标文件路径，包含图片对应的坐标信息。
        class_map (dict): 语言到类别id的映射字典。
        img_root (str): 图片根目录路径。
        input_size (int): 输入图片的尺寸，默认224。
        augment (bool): 是否进行数据增强，默认True。
    """
    def __init__(self, gt_file, coord_file, class_map, img_root, input_size=224, augment=True):
        self.samples = []  # 存储所有样本的信息
        self.class_map = class_map  # 语言到类别id的映射
        self.input_size = input_size  # 输入图片的尺寸
        self.augment = True;  # 是否进行数据增强

        # 同时打开标注文件和坐标文件，逐行读取
        with open(gt_file, encoding='utf-8') as gt_file, open(coord_file, encoding='utf-8') as coords_file:
            for gt_line, coords_line in zip(gt_file, coords_file):
                gt_parts = gt_line.strip().split(',')  # 解析标注行
                coords_part = coords_line.strip().split(',')  # 解析坐标行

                word_img = gt_parts[0]  # 图片文件名
                language_label = gt_parts[1]  # 语言标签
                coords = np.array(coords_part[1:-2], dtype=np.float32).reshape(4, 2)  # 解析坐标信息

                self.samples.append({
                    'word_path': os.path.join(img_root, word_img),
                    'coords': coords,
                    'language': language_label
                })